# Implement AdditivePhylogeny

[ba7b](https://rosalind.info/problems/ba7c/)

The following recursive algorithm, called AdditivePhylogeny, finds the simple tree fitting an n x n additive distance matrix D. We assume that you have already implemented a program Limb(D, j) that computes limb_length(j) for a leaf j based on the distance matrix D. Rather than selecting an arbitrary leaf j from Tree(D) for trimming, AdditivePhylogeny selects leaf n (corresponding to the last row and column of D).

    AdditivePhylogeny(D, n)
        if n = 2
            return the tree consisting of a single edge of length D1,2
        limb_length ← Limb(D, n)
        for j ← 1 to n - 1
            Dj,n ← Dj,n - limb_length
            Dn,j ← Dj,n
        (i,n,k) ← three leaves such that Di,k = Di,n + Dn,k
        x ← Di,n
        remove row n and column n from D
        T ← AdditivePhylogeny(D, n - 1)
        v ← the (potentially new) node in T at distance x from i on the path between i and k
        add leaf n back to T by creating a limb (v, n) of length limb_length
        return T

## Additive Phylogeny Problem

Construct the simple tree fitting an additive matrix.

    Given: 
n and a tab-delimited n x n additive matrix.

    Return: 
A weighted adjacency list for the simple tree fitting this matrix.

Note on formatting: The adjacency list must have consecutive integer node labels starting from 0. The n leaves must be labeled 0, 1, ..., n-1 in order of their appearance in the distance matrix. Labels for internal nodes may be labeled in any order but must start from n and increase consecutively.

In [17]:
import numpy as np

In [18]:
def get_limb_length(i, n, D):
    return int(min([D[i][k]+D[i][j]-D[j][k] for j in range(n) for k in range(n) if j!=k and k!=i and i!=j])/2)

In [19]:
def get_path(graph, v, from_node, to_node):
    v[from_node] = True
    for i, j in graph[from_node]:
        if v[i]:
            continue

        if i == to_node:
            return [(from_node, j), (to_node, 0)]
            
        path = get_path(graph, v, i, to_node)
        if path is not None:
            return [(from_node, j)] + path
    return

In [20]:
def delete_edge(T, from_node, to_node):
    for i, (neighbor, _) in enumerate(T[from_node]):
        if neighbor == to_node:
            break
    del T[from_node][i]

    for i, (neighbor, _) in enumerate(T[to_node]):
        if neighbor == from_node:
            break
    del T[to_node][i]

In [21]:
def insert_node(T, from_node, to_node, x, v, node):
    path = get_path(T, v, from_node, to_node)

    edge_length = path[0][1]
    distance = x

    n = 0 
    while distance >= edge_length:
        distance -= edge_length
        edge_length = path[n + 1][1]
        n += 1
        
    curr_node = path[n][0]
    next_node = path[n + 1][0]

    T[curr_node].append((node, distance))
    T[next_node].append((node, edge_length - distance))
    T[node] = [(curr_node, distance), (next_node, edge_length - distance)]

    delete_edge(T, curr_node, next_node)

In [22]:
def additive_phylogeny(D, n, node):
    if n == 2:
        length = D[0][1]
        return { 0:[(1, length)], 1:[(0, length)] }

    limbLength = get_limb_length(n - 1, n, D)
    for j in range(n - 1):
        D[j][n - 1] -= limbLength
        D[n - 1][j] = D[j][n - 1]

    for i in range(n-1):
        for k in range(i + 1, n - 1):
            if D[i][k] == D[i][n - 1] + D[k][n - 1]:
                x = D[i][n-1]
                from_node, to_node = i, k
                break
    
    T = additive_phylogeny(D, n - 1, node - 1)
    v = [False] * (2 * len(D))
    insert_node(T, from_node, to_node, x, v, node)

    T[n - 1] = [(node, limbLength)]
    T[node].append((n - 1, limbLength))

    return T

In [23]:
def print_graph(graph):
    graph = dict(sorted(graph.items()))
    for node in graph.keys():
        for edge in graph[node]:
            print ('{0}->{1}:{2}'.format(node,edge[0],edge[1]))

In [24]:
file = "rosalind_ba7c.txt" 
with open(file, 'r') as f:
    lines = f.readlines()
    n = int(lines[0])
    D = [[int(s) for s in line.split()] for line in lines[1:]]

graph = additive_phylogeny(D, n, 2 * len(D) - 3)
print_graph(graph)

0->21:289
1->26:62
2->20:323
3->22:825
4->29:527
5->23:447
6->24:823
7->25:310
8->34:305
9->27:670
10->28:315
11->29:260
12->30:760
13->31:575
14->32:330
15->33:783
16->35:168
17->35:171
18->36:293
19->37:539
20->2:323
20->23:648
20->26:202
21->0:289
21->24:182
21->30:709
22->3:825
22->25:422
22->29:933
23->20:648
23->5:447
23->28:670
24->21:182
24->6:823
24->33:448
25->22:422
25->7:310
25->37:946
26->1:62
26->20:202
26->31:832
27->9:670
27->32:803
27->36:869
28->23:670
28->10:315
28->30:325
29->4:527
29->22:933
29->11:260
30->28:325
30->21:709
30->12:760
31->26:832
31->13:575
31->34:730
32->27:803
32->14:330
32->37:117
33->24:448
33->15:783
33->36:158
34->8:305
34->31:730
34->35:530
35->34:530
35->16:168
35->17:171
36->27:869
36->33:158
36->18:293
37->25:946
37->32:117
37->19:539
